# Original question(s)

* How do I rename columns from an Excel file as {sheet_name}-{column_name}?

* How do I make it work for multiple excel sheets (all column names should be put together)?

# Let's create an example Excel file with multiple sheets

In the example Irene gave me all sheets had the same common index (e.g. a person id) with slightly different values (i.e. some information would be missing for an id)

In [1]:
import pandas as pd

df1 = pd.DataFrame({'first_name':['John', 'Daniel', 'Gabriela'],
                    'last_name':['Doe', 'Zimmermann', 'Mustermann']},
                   index=pd.Index([10, 20, 30], name='person_id'))

df2 = pd.DataFrame({'vehicle':['Supercar 200', 'Tesla Model X'],
                    'registration_number':['ABC-123', 'FOO-BAR']},
                   index=pd.Index([10, 20], name='person_id'))

df3 = pd.DataFrame({'favorite_dish':['Pizza', 'Spaghetti al pesto'],
                    'favorite_animal':['turtle', 'rooster']},
                   index=pd.Index([20, 30], name='person_id'))

# when we provide a path to df.to_excel it overwrites existing .xlsx files
# which means you cannot write to multiple sheets
# but instead of a path we can provide a pd.ExcelWriter object which allows this
writer = pd.ExcelWriter('example.xlsx')
df1.to_excel(writer, sheet_name='basic_infos')
df2.to_excel(writer, sheet_name='vehicles')
df3.to_excel(writer, sheet_name='likes')
writer.save() # don't forget to do this when you are done

print('sheet "basic_infos"')
display(df1)
print('sheet "vehicles"')
display(df2)
print('sheet "likes"')
df3

sheet "basic_infos"


,first_name,last_name
person_id,,
10,John,Doe
20,Daniel,Zimmermann
30,Gabriela,Mustermann


sheet "vehicles"


,vehicle,registration_number
person_id,,
10,Supercar 200,ABC-123
20,Tesla Model X,FOO-BAR


sheet "likes"


,favorite_dish,favorite_animal
person_id,,
20,Pizza,turtle
30,Spaghetti al pesto,rooster


# Solution

Note: `NaN` or `<NA>` indicates missing information in the resulting DataFrame

In [2]:
# let's a pd.ExcelFile object in order to get sheet names automatically
# (and we can also use this object to read sheets as we would with pd.read_excel)
xl = pd.ExcelFile('example.xlsx')

# get sheet names and prepare list to gather DataFrames
sheet_names = xl.sheet_names
dfs = []

# for each sheet name:
# 1) read it (get a pd.DataFrame)
# 2) rename columns (add sheet name as prefix). I did this in the simplest way I could think of
# 3) print the sheet name and the corresponding DataFrame
# 4) add the DataFrame corresponding to the sheet to the DataFrame list
for sheet_name in sheet_names:

    df = xl.parse(sheet_name=sheet_name, index_col='person_id') # set "person_id" as the index (important for concatenation)
    df.columns = sheet_name + '-' + df.columns.astype(str) # converts any non string columns to string (e.g. 123 -> "123")

    print(sheet_name)
    display(df)

    dfs.append(df)


# concatenate the list of DataFrames along the columns
df_all = pd.concat(objs=dfs,
                   axis='columns',
                   join='outer') # default, includes indices from all DataFrames (as some profiles have info missing)
df_all

basic_infos


,basic_infos-first_name,basic_infos-last_name
person_id,,
10,John,Doe
20,Daniel,Zimmermann
30,Gabriela,Mustermann


vehicles


,vehicles-vehicle,vehicles-registration_number
person_id,,
10,Supercar 200,ABC-123
20,Tesla Model X,FOO-BAR


likes


,likes-favorite_dish,likes-favorite_animal
person_id,,
20,Pizza,turtle
30,Spaghetti al pesto,rooster


,basic_infos-first_name,basic_infos-last_name,vehicles-vehicle,vehicles-registration_number,likes-favorite_dish,likes-favorite_animal
person_id,,,,,,
10,John,Doe,Supercar 200,ABC-123,NaN,NaN
20,Daniel,Zimmermann,Tesla Model X,FOO-BAR,Pizza,turtle
30,Gabriela,Mustermann,NaN,NaN,Spaghetti al pesto,rooster
